In [2]:
import pandas as pd
import numpy as np
import datetime
path = "task_1.csv"



In [43]:
#dateparse = lambda x: datetime.datetime.strptime(x, '%d-%m-%Y %H:%M:%S')
tmp = pd.read_csv(
    path,
    encoding = "cp1251",
    sep=";",
    on_bad_lines='skip',
    header=0,
    names=[
        "number",
        "type",
        "service",
        "priority",
        "status",
        "dt_create",
        "dt_solve",
        "dt_close",
        "firm",
        "group",
        "set",
        "client",
        "topic",
        "sla"
    ],

)


#####  1. Рассчитайте предварительные показатели для 1 квартала 2022 и подготовьте динамику для сравнения за предыдущие периоды

In [44]:

tmp[["dt_create", "dt_solve", "dt_close"]]  = tmp[["dt_create", "dt_solve", "dt_close"]].astype('datetime64')
df = tmp.loc[
        (tmp["type"] == "Инцидент") &
        (tmp["client"] != 'Мониторинг') &
        (tmp["sla"] == 'Да') &
        (tmp["service"].isin(["НОП", "TelcoCloud", "Платформа Цифровых Продуктов", "РТК-Экраны", "ЗСПД ЕГИСЗ", "ППЭ"])) &
        (tmp["firm"] == 'ДЭФИР') 
    ]
df

df['quoter'] = df['dt_solve'].dt.quarter






C:\Users\KurzanovAV\AppData\Local\Temp\ipykernel_13756\4246404460.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['quoter'] = df['dt_solve'].dt.quarter


In [64]:
df_pivot = pd.pivot_table(
    df[df['dt_solve'] >= pd.to_datetime('2021-07-01')], 
    values='number',  
    index='service', 
    columns='quoter', 
    aggfunc='count', 
)

df_pivot.apply(lambda x: x.div(x.sum()))

quoter,1,3,4
service,,,
TelcoCloud,0.018957,0.018450,0.021944
ЗСПД ЕГИСЗ,0.042654,0.044280,0.047022
НОП,0.867299,0.815498,0.846395
ППЭ,0.023697,0.062731,0.040752
Платформа Цифровых Продуктов,0.028436,0.011070,0.021944
РТК-Экраны,0.018957,0.047970,0.021944


##### 2. Рассчитайте, какая группа специалистов ДЭФИР решила больше всего обращений за 2 квартал 2021 года с учетом, что решившая группа остается заполненной в поле Назначенная группа.

In [63]:
df.loc[df['quoter'] == 2, ['number', 'group']].groupby('group').count()

,number
group,
Группа 1,53
Группа 2,67
Группа 3,57
Группа 4,50
Группа 5,65
